In [1]:
from regression_toolbox.model import Model, add_model, delete_model
from regression_toolbox.likelihood import Likelihood
from regression_toolbox.parameter import Parameter
from visualisation_toolbox.domain import Domain
from regression_toolbox.sampler import Sampler
from visualisation_toolbox.visualiser import Visualiser
from data_processing.sim_data_processor import SimDataProcessor
from data_processing.raw_data_processor import RawDataProcessor
import numpy as np
import pandas as pd
import os

os.chdir('/PhD_project/')

/build/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
report_title = 'Simulation run with noise level 1 on line model'

seed = 42
# Set the seed for numpy
np.random.seed(seed)


# PARAMETERS
n_samples = 10            # Number of data samples (per dimension for 2D/3D)
noise_scale = 1            # Noise scale for data generation
num_warmup = 500            # Number of warmup steps for MCMC
num_samples = 1000         # Number of MCMC samples
model_name = 'line'    

delete_model(model_name)
add_model(model_name, 'a*x + b', ['x'], ['y'], ['a', 'b'])

# TRUE VALUES FOR PARAMETERS
true_params = {'a': 2.5, 'b': 1.5}#, 'c': 3.5, 'd': 4.5, 'e': 5.5, 'f': 6.5}

sim_model = Model('line').add_fixed_model_param('a', true_params['a']).add_fixed_model_param('b',true_params['b'])
sim_domain = Domain('one_D').add_domain_param('x_min', 0.1).add_domain_param('x_max', 10).add_domain_param('resolution', n_samples)
sim_data_processor = SimDataProcessor('sim_data_line', sim_model, sim_domain, noise_dist='gaussian', noise_level=noise_scale, train_test_split=1.0)

model = Model('line')
likelihood = Likelihood('gaussian')
a = Parameter(name='a', prior_select='gaussian').add_prior_param('mu', 1).add_prior_param('sigma', 10)
b = Parameter(name='b', prior_select='gaussian').add_prior_param('mu', 1).add_prior_param('sigma', 1)
sigma = Parameter(name='sigma', prior_select='exponential').add_prior_param('rate', 1)
inference_params = pd.Series({'a': a, 'b': b, 'sigma': sigma})

sampler = Sampler(inference_params, model, likelihood, sim_data_processor, n_samples = num_samples, n_chains=1)
sampler.sample_all()
print(sampler.data_processor.processed_data)
print(sampler.testing_data)


visualiser = Visualiser(sampler)
visualiser.get_traceplots()

visualiser.get_summary()

visualiser.plot_prior('a', [0,10])
visualiser.plot_prior('b', [0,10])
visualiser.plot_prior('sigma', [0,10])

visualiser.plot_posterior('a', [0,10])
visualiser.plot_posterior('b', [0,10])
visualiser.plot_posterior('sigma', [0,10])

visualiser.generate_report(report_title)
visualiser.embed_report()


Data loaded from /PhD_project/data/processed_sim_data/sim_data_line


sample: 100%|██████████| 1500/1500 [00:01<00:00, 1204.25it/s, 23 steps of size 4.28e-01. acc. prob=0.90]


Samples saved to /PhD_project/results/inference_results/sim_data_line/general_instances/instance_2
   Unnamed: 0     x          y  y_true
0           0   0.1   2.246714    1.75
1           1   1.2   4.361736    4.50
2           2   2.3   7.897688    7.25
3           3   3.4  11.523029   10.00
4           4   4.5  12.515846   12.75
5           5   5.6  15.265863   15.50
6           6   6.7  19.829212   18.25
7           7   7.8  21.767435   21.00
8           8   8.9  23.280525   23.75
9           9  10.0  27.042560   26.50
   Unnamed: 0     x          y  y_true
0           0   0.1   2.246714    1.75
1           1   1.2   4.361736    4.50
2           2   2.3   7.897688    7.25
3           3   3.4  11.523029   10.00
4           4   4.5  12.515846   12.75
5           5   5.6  15.265863   15.50
6           6   6.7  19.829212   18.25
7           7   7.8  21.767435   21.00
8           8   8.9  23.280525   23.75
9           9  10.0  27.042560   26.50
